**This notebook runs the  pipeline to identify DEI-related practice and outcome terms and their co-occurrences in a corpus of texts. It will output a CSV.**

The files `'practice_outcome_term_dict.csv'` and `'DEI_context_term_dict.csv'` will need to be downloaded and placed in this notebook's working directory.

*For your information:*

## (RUN ENTIRE SECTION AS IS): Imports

In [ ]:
# Ensures the tqdm package is up-to-date as necessary
!pip install "tqdm>=4.9.0"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install pycountry

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 38.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pycountry: filename=pycountry-22.3.5-py2.py3-none-any.whl size=10681845 sha256=897c011cde119fff1f52903143d397235f4692a9887a74fd8a7815704be56671
  Stored in directory: /root/.cache/pip/wheels/e2/aa/0f/c224e473b464387170b83ca7c66947b4a7e33e8d903a679748
Successfully built pycountry


In [ ]:
import datetime
import numpy as np
import os
import sys
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns
import random
import glob

from tqdm import tqdm
import pycountry

import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly.express as px

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.tokenize import word_tokenize

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
import itertools
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
import tensorflow as tf

from nltk import pos_tag
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('wordnet')
from nltk.corpus import wordnet
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
from nltk import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [ ]:
# VADER
!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 KB 3.4 MB/s eta 0:00:00


## (SLIGHT MODIFICATION): Mount your drive
- MODIFY-1: make sure you are in your working directory

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
cd '/content/drive/MyDrive/Work/Rights-Colab-SS/Github Demo'

/content/drive/MyDrive/Work/Rights-Colab-SS/Github Demo


In [ ]:
#MODIFY-1: change the filepath to your working directory
# NOTE: Make sure there is an output of the filepath
# otherwise you may have not actually moved into the working directory
!cd '/content/drive/MyDrive/your-working-directory'

## (RUN ENTIRE SECTION AS IS): Helper Functions

### Create Pattern Functions

In [ ]:
# Functions to create patterns of practice/outcome terms

def create_pattern_2(buck1, buck2, rangelist):
  '''
  Creates a pattern that checks if at least one search word in each of the 2
  buckets of search words are 1) within a certain number of words of each other
  and 2) as a phrase, within a certain number of words of a DEI context word
  in a text (context window).

  Parameters
  ----------
  buck1 : list of strings; len(buck1)>=1
      The strings are search words in regex format
  buck2 : list of strings; len(buck2)>=1
      The strings are search words in regex format
  rangelist : list of 2 ints of form [a,b]
      a = context window between buck1 and buck2 search words
      b = context window between buck1-buck2 phrase and DEI context word

  Returns
  -------
  [[buck1, buck2], rangelist] : formatted list of the parameters
  '''
  return [[buck1, buck2], rangelist]

def create_pattern_3(buck1, buck2, buck3, rangelist):
  '''
  Creates a pattern that checks if at least one search word in each of the 3
  buckets of search words are 1) within a certain number of words of each other
  and 2) as a phrase, within a certain number of words of a DEI context word
  in a text (context window).

  Parameters
  ----------
  buck1 : list of strings; len(buck1)>=1
      The strings are search words in regex format
  buck2 : list of strings; len(buck2)>=1
      The strings are search words in regex format
  buck3 : list of strings; len(buck3)>=1
      The strings are search words in regex format
  rangelist : list of 4 ints of form [a_1_2,a_2_3,a_1_3,b]
      a_1_2 = context window between buck1 and buck2 search words
      a_2_3 = context window between buck2 and buck3 search words
      a_1_3 = context window between buck1 and buck3 search words
      b = context window between buck1-buck2-buck3 phrase and DEI context word

  Returns
  -------
  [[buck1, buck2, buck3], rangelist] : formatted list of the parameters
  '''
  return [[buck1, buck2, buck3], rangelist]

### Sentiment Analysis Functions

In [ ]:
# Functions used for generate_sentiment (copy and pasted from above)

# remove punctuation and numbers
def clean_data(text):
  text = re.sub('[^A-Za-z]+', ' ', text)
  return text

# Tokenize, POS tagging, Stopwords removal
pos_dict = {'J':wordnet.ADJ, 'V':wordnet.VERB, 'N':wordnet.NOUN, 'R':wordnet.ADV}
def token_stop_pos(text):
  tags = pos_tag(word_tokenize(text))
  new_list = []
  for word, tag in tags:
    if word.lower() not in set(stopwords.words('english')):
      new_list.append(tuple([word, pos_dict.get(tag[0])])) # tag[0] corresponds to the first letter of the tag
  return new_list

# Lemmatize function
def lemmatize(pos_data):
  lemma_rew = " "
  for word, pos in pos_data:
    if not pos:
      lemma = word
      lemma_rew = lemma_rew + " " + lemma
    else:
      lemma = wordnet_lemmatizer.lemmatize(word, pos=pos)
      lemma_rew = lemma_rew + " " + lemma
  return lemma_rew

# stemming function
def stem_text(text):
  stemmed = " "
  text = text.split()
  for word in text:
    stem = ps.stem(word)
    stemmed = stemmed + " " + stem
  return stemmed

# Sentiment function for VADER
analyzer = SentimentIntensityAnalyzer()
def vader_sentiment(text):
  vs = analyzer.polarity_scores(text)
  return vs['compound']

# Custom rule-based sentiment function

pos_words = ['divers', 'inclus', 'equal', 'achiev', 'commit', 'advanc', 'excel', 'empow']
neg_words = ['discrimin', 'lawsuit', 'harrass', 'racial', 'inequ', 'sue', 'disabililti', 'protest', 'against',
             'complaint', 'abus', 'misconduct', 'harass', 'racism', 'charg', 'gap']

def DEI_sentiment(stem_text):
  pos_score = 0
  neg_score = 0
  for word in pos_words:
    if word in stem_text:
      pos_score = pos_score + 1
  for word in neg_words:
    if word in stem_text:
      neg_score = neg_score + 1
  if(pos_score == neg_score):
    return 0
  elif(pos_score < neg_score):
    return -1
  else:
    return 1

# calculate absolute scores
def calc_vader(num):
  if num >= 0.7:
    return 1
  elif num < -0.7:
    return -1
  else:
    return 0

In [ ]:
# text should be a string
def generate_sentiment(text):
  cleaned_text = clean_data(text)
  processed_text = lemmatize(token_stop_pos(cleaned_text))
  stemmed_text = stem_text(cleaned_text)

  if 'leave' in text:
    vader_score = 0
  else:
    vader_score = calc_vader(vader_sentiment(processed_text))

  if(vader_score == 0):
    return DEI_sentiment(stemmed_text)
  else:
    return vader_score

### Find Practice/Outcome Terms Functions

In [ ]:
def check_empty_regex(buckets,word_ind_dict):
  edited_buckets = []
  for buck in buckets:
    edited = [regex for regex in buck if len(word_ind_dict[regex])!=0]
    edited_buckets.append(edited)
  return edited_buckets

In [ ]:
# Functions to find practice/outcome terms in a text

def word_indices(input_str, search_word_lst, DEI_contx_list):
  '''
  Finds the all indices (by word) of every occurence of every search word for
  practice/outcome terms and DEI context terms.

  Parameters
  ----------
  input_str : string
      This is the input text you are searching through.
  search_word_lst : list of strings
      The strings are the practice/outcome term pattern search words in regex format
  DEI_contx_list : list of strings
      The strings are the DEI context search words in regex format

  Returns
  -------
  word_ind_dict : dict of indices where each search word was found
      key: string of search word in regex format
      value: list of indices where that search word was found in the text

      Example: dict with 4 search words
      word_ind_dict = {'apprentice': [54],
                       'female': [0, 17, 21],
                       'program': [27],
                       'wom(e|a)n': [4]}
  '''
  # Build dict with all words as keys w/ empty lists as values
  total_list = search_word_lst + DEI_contx_list
  li = [(i,[]) for i in total_list]
  word_ind_dict = {}
  for j in li:
    word_ind_dict[j[0]] = j[1]

  # Fill empty lists with index matches
  for w in total_list:
    for match in re.finditer(w, str(input_str)):
      before_str = word_tokenize(input_str[:match.start()])
      word_ind_dict[w].append(len(before_str))

  return word_ind_dict

def check_cooccur(word_ind_dict, terms_dict, DEI_contx_list,input_str):
  '''
  Finds every instance of all practice/outcome terms (as pattern + DEI context
  word) in a text. Uses a dict of indices where each search word is found in the
  text, the output of function word_indices().

  Parameters
  ----------
  word_ind_dict : dict of indices where each search word is found in a text
      Output of function word_indices()
      key: string of search word in regex format
      value: list of indices where that search word was found in the text
  terms_dict : dict of practice/outcome term patterns
      key: name of practice/outcome term
      value: list of patterns for that term; [p0,p1...]
  DEI_contx_list : list of strings
      The strings are the DEI context search words in regex format

  Returns
  -------
  flagged_terms : dict of every instance of all practice/outcome terms
      key: name of practice/outcome term
      value: list of instances with each instance being a list of search words
      in regex format and the context window used to flag them

      Example: dict with 1 practice term found in 4 instances
      {'program-retain': [[('program', 'retain', 'talent', 'wom(e|a)n'), [10, 10, 10, 30]],
                          [('program', 'retain', 'talent', 'wom(e|a)n'), [10, 10, 10, 30]],
                          [('program', 'retain', 'talent', 'female'), [10, 10, 10, 30]],
                          [('program', 'apprenticeship', 'gender'), [4, 50]]]}
  '''
  flagged_terms = {}

  # Iterate through all of the terms and their patterns
  for key, value in terms_dict.items():
    is_match = 0
    match_pattern = []

    # Iterate through each pattern of a term
    for pattern in value: # pattern = [[[buck1], [buck2]], [a,b]]
      combo_buck = check_empty_regex(pattern[0]+[DEI_contx_list],word_ind_dict) # combo_buck = [[buck1], [buck2], [DEI_contx_list]]
      #combo_buck = pattern[0] + [DEI_contx_list] # combo_buck = [[buck1], [buck2], [DEI_contx_list]]

      has_empty = any(len(c_b) == 0 for c_b in combo_buck)
      if has_empty is False:

        combos = list(itertools.product(*combo_buck)) # List of combos of 1-ea word + 1 DEI context term, per pattern
        # Iterate through each possible search word combo for a pattern
        for c in combos: # c = ('lawsuit', 'discriminat', 'wom(e|a)n')

          # Collect indices for each search word
          ind_list = [] # List of indices-lists for each word in combo
          for w in c: # w = 'lawsuit'
            if w in word_ind_dict:
              ind_list.append(word_ind_dict[w]) # [[2,56],[3],[45,23,12]]

          # Check if ind_list has enough lists (every word in combo is found in text)
          if len(ind_list) == len(c):
            # Check if indices are in range
            combos_inds = list(itertools.product(*ind_list)) # All possible combos of indices: combos_inds = [(24, 53, 20), (24, 53, 28)]

            # Iterate through each combo of indices
            for c_i in combos_inds:

              # if is a 2-bucket pattern, c_i = (24, 53, 20)
              if len(pattern[1]) == 2: # pattern[1] = [a,b] -> rangelist
                subrange = [c_i[0],c_i[1]]
                subrange.sort()
                # if DEI context word is within term phrase buffer zone (24-(b+1) <= 20 <= 50+(b+1))
                # AND search words are within term pattern context window (|24-53|-1 <= a)
                context_wind_start = max(0,subrange[0]-(pattern[1][1]+1))
                context_wind_end = min(subrange[1]+(pattern[1][1]+1),len(word_tokenize(input_str)))
                if (context_wind_start <= c_i[2] <= context_wind_end) and (abs(c_i[0] - c_i[1])-1 <= pattern[1][0]):
                  is_match = 1

                  input_token = word_tokenize(input_str)
                  bubble_token = input_token[max(0,min(c_i)-10):min(max(c_i)+10,len(input_token)-2)+1]
                  bubble = ' '.join(bubble_token)
                  sentiment_score = generate_sentiment(bubble)

                  match_pattern.append([c,pattern[1],sentiment_score])

              # if is a 3-bucket pattern, c_i = (50, 34, 60, 61)
              elif len(pattern[1]) == 4: # pattern[1] = [a_1_2,a_2_3,a_1_3,b] -> rangelist
                subrange = [c_i[0],c_i[1],c_i[2]]
                subrange.sort()
                # if DEI context word is within term phrase buffer zone (34-(b+1) <= 61 <= 60+(b+1))
                # AND search words are within term pattern context windows (|50-34|-1 <= a_1_2 AND |34-60|-1 <= a_2_3 AND |50-60|-1 <= a_1_3)
                context_wind_start = max(0,subrange[0]-(pattern[1][3]+1))
                context_wind_end = min(subrange[2]+(pattern[1][3]+1),len(word_tokenize(input_str)))
                if (context_wind_start <= c_i[3] <= context_wind_end) and (abs(c_i[0] - c_i[1])-1 <= pattern[1][0]) and (abs(c_i[1] - c_i[2])-1 <= pattern[1][1]) and (abs(c_i[0] - c_i[2])-1 <= pattern[1][2]):
                  is_match = 1

                  input_token = word_tokenize(input_str)
                  bubble_token = input_token[max(0,min(c_i)-10):min(max(c_i)+10,len(input_token)-2)+1]
                  bubble = ' '.join(bubble_token)
                  sentiment_score = generate_sentiment(bubble)

                  match_pattern.append([c,pattern[1],sentiment_score])

              else:
                print("wrong range length: ", pattern[1], pattern[0])
                is_match = -1

    # If there is at least one instance found for the term, insert the match pattern into flagged_terms dict
    if is_match == 1:
      flagged_terms[key] = match_pattern

  return flagged_terms

### Find Modifier Terms Function



In [ ]:
def get_modifier_data(input_string, modifier_dict):
  small_ind_data = []
  small_ind_col_name = []
  big_ind_data = []
  big_ind_col_name = []

  for name, value in modifier_dict.items():
    indicator_col_name = name+'_MODIFIER_'+value[1]
    big_ind_col_name.append(indicator_col_name)

    per_IND_data = []
    for keyword in value[0]:
      small_ind_col_name.append(f'{keyword}_MOD')

      keyword_regex = re.compile(f'(?<![^ .,?!;]){keyword}')
      if keyword_regex.search(str(input_string)):
        per_IND_data.append(1)
      else:
        per_IND_data.append(0)

    small_ind_data = small_ind_data + per_IND_data
    if sum(per_IND_data) > 0:
      big_ind_data.append(1)
    else:
      big_ind_data.append(0)

  return small_ind_data, small_ind_col_name, big_ind_data, big_ind_col_name

###Pipeline Text Search Function

In [ ]:
def check_columns(row, text_column_name):
    input_string = row[text_column_name]
    word_index_dict = word_indices(input_string, search_words_list, DEI_context_list)
    flag_term_dict = check_cooccur(word_index_dict, terms_dict, DEI_context_list, input_string)

    flagged_terms = []
    OUTCOME_list = ''
    PRACTICE_list = ''
    DEI_terms_list = ''

    RRP_bool = 0 # Recruitment/Retention/Promotion-Rates
    WVP_bool = 0 # Worker-Voice/Protection
    DEISG_bool = 0 # DEI-Strategy/Governance
    MBOD_bool = 0 # Management/Board/Owner-Demographics
    WD_bool = 0 # Workforce-Development
    PE_bool = 0 # Pay-Equity
    SC_bool = 0 # Supply-Chain
    BS_bool = 0 # Benefits/Services
    CI_bool = 0 # Customer-Inclusiveness
    WSP_bool = 0 # Worker-Security/Precarity
    CA_bool = 0 # Community-Action
    PA_bool = 0 # Public-Advocacy
    GO_bool = 0 # General-Outcome
    GP_bool = 0 # General-Practice

    add_columns = ['PRACTICE_TERMS_FOUND','OUTCOME_TERMS_FOUND','DEI-CONTEXT_TERMS_FOUND', 'ANY_RRP_TOPIC','ANY_WVP_TOPIC','ANY_DEISG_TOPIC','ANY_MBOD_TOPIC','ANY_WD_TOPIC','ANY_PE_TOPIC','ANY_SC_TOPIC','ANY_BS_TOPIC','ANY_CI_TOPIC','ANY_WSP_TOPIC','ANY_CA_TOPIC','ANY_PA_TOPIC','ANY_GO_TOPIC','ANY_GP_TOPIC']
    series_columns = []
    for term in terms_list:
      indicator_col_name = "{}_{}_{}".format(term, terms_category[term][1], terms_category[term][0])
      add_columns.append(indicator_col_name)
      series_columns.append(term)
    for dt in DEI_context_list:
      indicator_col_name = "{}_{}_{}".format(dt, 'DEI-CONTEXT', DEI_context_dict[dt])
      add_columns.append(indicator_col_name)
      series_columns.append(dt)

    for key, value in flag_term_dict.items():
      flagged_terms.append(key)

      value_unique_str = list(set([str(v) for v in value]))
      DEI_unique = list(set([instance[0][len(instance[0])-1] for instance in value]))
      flagged_terms = flagged_terms + DEI_unique

      if terms_category[key][1] == 'OUTCOME':
        OUTCOME_list = OUTCOME_list + key + ' ('+ terms_category[key][0] + '): '+ '\n'.join(value_unique_str) + '\n\n'
      elif terms_category[key][1] == 'PRACTICE':
        PRACTICE_list = PRACTICE_list + key + ' ('+ terms_category[key][0] + '): '+ '\n'.join(value_unique_str) + '\n\n'

      if terms_category[key][0] == 'Recruitment/Retention/Promotion-Rates':
        RRP_bool = 1
      elif terms_category[key][0] == 'Worker-Voice/Protection':
        WVP_bool = 1
      elif terms_category[key][0] == 'DEI-Strategy/Governance':
        DEISG_bool = 1
      elif terms_category[key][0] == 'Management/Board/Owner-Demographics':
        MBOD_bool = 1
      elif terms_category[key][0] == 'Workforce-Development':
        WD_bool = 1
      elif terms_category[key][0] == 'Pay-Equity':
        PE_bool = 1
      elif terms_category[key][0] == 'Supply-Chain':
        SC_bool = 1
      elif terms_category[key][0] == 'Benefits/Services':
        BS_bool = 1
      elif terms_category[key][0] == 'Customer-Inclusiveness':
        CI_bool = 1
      elif terms_category[key][0] == 'Worker-Security/Precarity':
        WSP_bool = 1
      elif terms_category[key][0] == 'Community-Action':
        CA_bool = 1
      elif terms_category[key][0] == 'Public-Advocacy':
        PA_bool = 1
      elif terms_category[key][0] == 'General-Outcome':
        GO_bool = 1
      elif terms_category[key][0] == 'General-Practice':
        GP_bool = 1

      for DEI_word in DEI_unique:
        DEI_terms_list = DEI_terms_list + DEI_word + ' ('+ DEI_context_dict[DEI_word] + ') '+ ' ['+ key + '], \n'

    flagged_terms_ind = []
    for s in series_columns:
      if s in flagged_terms:
        flagged_terms_ind.append(1)
      else:
        flagged_terms_ind.append(0)

    pre_list = [PRACTICE_list, OUTCOME_list, DEI_terms_list, RRP_bool, WVP_bool, DEISG_bool, MBOD_bool, WD_bool, PE_bool, SC_bool, BS_bool, CI_bool, WSP_bool, CA_bool, PA_bool, GO_bool, GP_bool]
    series_data = pre_list + flagged_terms_ind

    # Find modifier terms
    small_ind_data, small_ind_col_name, big_ind_data, big_ind_col_name = get_modifier_data(input_string, modifier_dict)

    ind_summary_col_name = ['MODIFIER_TERMS_FOUND']
    pre_list = [big_ind_col_name[i].split('_')[0] + ' ('+big_ind_col_name[i].split('_')[2]+')' for i in range(len(big_ind_col_name)) if big_ind_data[i] >= 1]
    ind_str = ''
    for p in pre_list:
      ind_str = ind_str+p+'\n'
    ind_summary_data = [ind_str]

    # Update column name list and data list with country and modifier data
    #series_data = series_data + ind_summary_data + ctry_data + big_ind_data + small_ind_data
    #add_columns = add_columns + ind_summary_col_name + ctry_col_name + big_ind_col_name + small_ind_col_name
    series_data = series_data + ind_summary_data + big_ind_data + small_ind_data
    add_columns = add_columns + ind_summary_col_name + big_ind_col_name + small_ind_col_name

    final_series = pd.Series(data=series_data, index =add_columns)

    return final_series

## (SLIGHT MODIFICATION) Term dictionaries and lists
The files `'practice_outcome_term_dict.csv'` and `'DEI_context_term_dict.csv'` will be loaded from the working directory, where they should be placed. Change the filepaths as necessary.

- MODIFY-1.1 & MODIFY 1.2: make sure the 2 CSVs are in your working directory

*For your information:*
- *practice terms are a thing you do: managing, hiring, training, setting up programs (gerunds)*
- *outcome terms: everything else -> that results from these actions*

### Practice/Outcome Terms

In [ ]:
keywords_csv_path = 'practice_outcome_term_dict.csv' #MODIFY-1.1: adjust filepath as necessary

In [ ]:
# Some variables that are useful to toggle later
MAX_EXPECTED_PATTERNS = 4  # This is based on the number of pattern columns you have currently

first_colname_keywords = "Framework Topic"
term_colname = "Term"
term_category_colname = "Framework Topic"
term_pattern_status_colname = "Status"
term_pattern_colname = "Patterns"
term_type_colname = "Term Type"

# Read in practice/outcome terms csv sheet
df_keywords = pd.read_csv(keywords_csv_path)

# Change names of pattern columns (previously nan)
pattern_cols_start_idx = list(df_keywords.columns).index('Patterns')

updated_colnames = []
for colname in list(df_keywords.columns)[:pattern_cols_start_idx]:
    updated_colnames.append(colname)

pattern_col_counter = 1
for colname in list(df_keywords.columns)[pattern_cols_start_idx: pattern_cols_start_idx+MAX_EXPECTED_PATTERNS]:
    updated_colnames.append(f"Patterns {pattern_col_counter}")
    pattern_col_counter += 1
del pattern_col_counter

for colname in list(df_keywords.columns)[pattern_cols_start_idx+MAX_EXPECTED_PATTERNS:]:
    updated_colnames.append(colname)

df_keywords.columns = updated_colnames

pattern_cols = [f"Patterns {i}" for i in range(1, MAX_EXPECTED_PATTERNS + 1)]

# Iterate through keywords
terms_dict = dict()
terms_category = dict()

for i, row in df_keywords.iterrows():

    # Check if it's a NaN row
    status = row[term_pattern_status_colname]
    if pd.isna(status) and pd.isna(row[term_colname]):
        continue

    # Check if term patterns are complete
    if "In progress" in str(status) or str(status) == "" or pd.isna(status):
        continue

    # Clean patterns from columns to prepare to add to dict
    print(i)
    print(row[term_colname])
    print(status)
    patterns_READY_TO_ADD = []

    for p_col in pattern_cols:
        patterns = row[p_col]
        if patterns == "-" or pd.isna(patterns):
            continue

        delim = "],"
        delim_add_back = "]"
        pattern_lst = [e+delim_add_back for e in row[p_col].split(delim) if e]
        pattern_lst[-1] = pattern_lst[-1][:-1]
        # pattern_lst

        pattern_lst_update_1 = []
        for elem in pattern_lst:
            pattern_lst_update_1.append(elem.strip('][').split(', '))
        # pattern_lst_update_1

        pattern_lst_update_2 = []
        for elem in pattern_lst_update_1:
            # pattern_lst_update_2.append([sub_elem.replace("\"\'", "\"").replace("\'\"", "\"") for sub_elem in elem])
            pattern_lst_update_2.append([sub_elem.replace("'", "") for sub_elem in elem])
        # pattern_lst_update_2

        # Create and add patterns to patterns_READY_TO_ADD
        # try:
        pattern_lst_update_2[-1] = [int(i) for i in pattern_lst_update_2[-1][0].split(',')]

        if len(pattern_lst_update_2) == 3:
            ptn = create_pattern_2(pattern_lst_update_2[0], pattern_lst_update_2[1], pattern_lst_update_2[2])
        elif len(pattern_lst_update_2) == 4:
            ptn = create_pattern_3(pattern_lst_update_2[0], pattern_lst_update_2[1], pattern_lst_update_2[2], pattern_lst_update_2[3])


        patterns_READY_TO_ADD.append(ptn)

        # except ValueError:
        #     print(pattern_lst_update_2)
        #     print(pattern_lst_update_2[-1])
        #     break


    terms_dict[row[term_colname]] = patterns_READY_TO_ADD
    terms_category[row[term_colname]] = (row[term_category_colname], row[term_type_colname].upper())


0
life insurance
Finalized (SET) pattern development
1
disability coverage
Finalized (SET) pattern development
2
retirement provision
Finalized (SET) pattern development
3
stock ownership
Finalized (SET) pattern development
4
equal benefits
Finalized (SET) pattern development
5
family leave
Finalized (SET) pattern development
6
leave-number of weeks paid/unpaid
Finalized (SET) pattern development
7
leave-percent of salary covered
Finalized (SET) pattern development
8
leave-impact on bonus/carry/vesting
Finalized (SET) pattern development
9
progams/initiatives/policy-return to the workplace
Finalized (SET) pattern development
10
track composition-benefits
Finalized (SET) pattern development
11
report composition-benefits
Finalized (SET) pattern development
12
programs/initiatives/policy-work-life balance
Finalized (SET) pattern development
13
incentivise equal sharing of caring responsibilities
Finalized (SET) pattern development
14
family-building coverage
Finalized (SET) pattern devel

In [ ]:
# View terms_dict, which maps the practice/outcome term to its term pattern(s)
terms_dict

{'life insurance': [[[[' provid(e|ing)',
     ' (giv(e|ing)|gave )',
     ' offer',
     ' inclu(d|sive)',
     ' add(ed|ing|s)? '],
    ['life insurance']],
   [5, 15]]],
 'disability coverage': [[[[' provid(e|ing)',
     ' (giv(e|ing)|gave )',
     ' offer',
     ' inclu(d|sive)',
     ' add(ed|ing|s)? '],
    ['insurance', 'coverage'],
    [' disab', 'invalidity']],
   [7, 5, 7, 15]]],
 'retirement provision': [[[[' provid(e|ing)',
     ' (giv(e|ing)|gave )',
     ' offer',
     ' inclu(d|sive)',
     ' add(ed|ing|s)? '],
    ['provision'],
    [' retir(e|ing)']],
   [7, 5, 7, 15]]],
 'stock ownership': [[[[' stock(s)? ', ' share(s)? '],
    [' own',
     ' grant',
     ' provid(e|ing)',
     ' (giv(e|ing)|gave )',
     ' offer',
     ' inclu(d|sive)',
     ' add(ed|ing|s)? '],
    ['employ', 'work(er|force|( )?place)', 'contractor']],
   [6, 8, 10, 15]]],
 'equal benefits': [[[['equal', 'equivalent', ' same', 'identical'],
    [' leave', 'treatment', 'benefits', 'coverage']],
   [5

In [ ]:
# View terms_category, which maps the practice/outcome term to its theme and term type
terms_category

{'life insurance': ('Benefits/Services', 'PRACTICE'),
 'disability coverage': ('Benefits/Services', 'PRACTICE'),
 'retirement provision': ('Benefits/Services', 'PRACTICE'),
 'stock ownership': ('Benefits/Services', 'PRACTICE'),
 'equal benefits': ('Benefits/Services', 'PRACTICE'),
 'family leave': ('Benefits/Services', 'PRACTICE'),
 'leave-number of weeks paid/unpaid': ('Benefits/Services', 'PRACTICE'),
 'leave-percent of salary covered': ('Benefits/Services', 'PRACTICE'),
 'leave-impact on bonus/carry/vesting': ('Benefits/Services', 'PRACTICE'),
 'progams/initiatives/policy-return to the workplace': ('Benefits/Services',
  'PRACTICE'),
 'track composition-benefits': ('Benefits/Services', 'PRACTICE'),
 'report composition-benefits': ('Benefits/Services', 'PRACTICE'),
 'programs/initiatives/policy-work-life balance': ('Benefits/Services',
  'PRACTICE'),
 'incentivise equal sharing of caring responsibilities': ('Benefits/Services',
  'PRACTICE'),
 'family-building coverage': ('Benefits/S

In [ ]:
search_words_list = []
terms_list = []
for key, val in terms_dict.items():
  terms_list.append(key)
  for pattern in val:
    bucks_list = pattern[0]
    for bucket in bucks_list:
      for word in bucket:
        search_words_list.append(word)

search_words_list = list(set(search_words_list))
terms_list = list(set(terms_list))

### DEI Context Terms

In [ ]:
DEI_context_dict_df = pd.read_csv("DEI_context_term_dict.csv") #MODIFY-1.2: adjust filepath as necessary

In [ ]:
DEI_context_dict = DEI_context_dict_df.set_index('REGEX Term').to_dict()['Category']

In [ ]:
# View DEI_context_dict, which maps the DEI context term to its term category
DEI_context_dict

{'ethnic': 'ethnicity',
 '(disab(i|l)| able-bod)': '(dis)ability',
 '(marital status|married)': 'marital status',
 ' bias': 'bias',
 'religio': 'religio',
 'inclusiv': 'inclusive',
 'divers': 'diverse',
 ' race( |,|.)': 'race',
 'racism': 'race',
 'racist': 'race',
 'racial': 'race',
 'bipoc': 'race',
 'people of colo(u)?r': 'race',
 'blackface': 'race',
 '(black|african american)': 'race',
 'white': 'race',
 'asian': 'race',
 'latin(o|a|x)': 'race',
 'hispanic': 'ethnicity',
 '(indigenous|native(s| (america|population|communit|govern|reservation))|american indian|amer(-)?indian|first nations|trib(al|e)|aborigin)': 'race',
 '(environmental human rights defender|ehrd)': 'advocate',
 'working (famil|parent|mother|mom|father|dad)': 'familial status',
 'military status': 'military status',
 'veteran': 'military status',
 '(service|guard|reserve) member': 'military status',
 '(active|military) duty': 'military status',
 'minorities': 'minorit',
 'minority group': 'minorit',
 'lgbt': 'LGBT',

In [ ]:
DEI_context_list = []
for key, val in DEI_context_dict.items():
  DEI_context_list.append(key)

DEI_context_list = list(set(DEI_context_list))

### Modifier Terms

In [ ]:
modifier_dict = {
    'full-time employment':(['full( |-)?time(.*)(employ|work|labo(u)?r)','(employ|work|labo(u)?r)(.*)full( |-)?time'],'EmploymentType'),
    'part-time employment':(['part( |-)?time(.*)(employ|work|labo(u)?r)','(employ|work|labo(u)?r)(.*)part( |-)?time'],'EmploymentType'),
    'permanent employment':(['permanent(.*)(employ|work|labo(u)?r)','(employ|work|labo(u)?r)(.*)permanent'],'EmploymentType'),
    'temporary employment':(['temporary(.*)(employ|work|labo(u)?r)','(employ|work|labo(u)?r)(.*)temporary'],'EmploymentType'),
    'zero-hour employment':(['zero( |-)?hour(.*)(employ|work|labo(u)?r)','(employ|work|labo(u)?r)(.*)zero( |-)?hour'],'EmploymentType'),
    'non-guaranteed hours employees':(['non( |-)?guarantee(.*)(employ|work|labo(u)?r)','(employ|work|labo(u)?r)(.*)non( |-)?guarantee'],'EmploymentType'),
    'human capital':(['human capital'],'HumanCapital'),
    'invest in people':(['invest in people','invest in a(n)? (person|individual)'],'HumanCapital'),
    'senior management/executive':(['president', ' vp ', 'c-level', 'partner( |-)level', '(firm|name|managing|principal) partner', 'director', 'chair(man|woman|person)', 'chair of', 'principals', '(senior|top) (leader|manag)', 'executive'],'CompanyEntity'),
    'board of directors/executive board':(['board of (director|executive)', '(director|executive) board'],'CompanyEntity'),
    'nonexecutive/supervisory board':(['board of (supervisor|non( |-)?executive)', '(supervisory|non( |-)?executive) board'],'CompanyEntity'),
    'C-suite':(['c-suite', 'c-level', ' ceo ', ' cto ', ' cfo ' , ' coo ', ' cio '],'CompanyEntity'),
    'investment committee/team':(['investment (committee|team)'],'CompanyEntity'),
    'stockholder/shareholder':(['(share|stock)holder'],'CompanyEntity'),
    'professionals':(['associates', 'analysts', 'professionals', 'entrepreneurs in residence'],'CompanyEntity'),
    'direct operations workforce':(['direct operations (employ|work|labo(u)?r)', 'floor(.*)(employ|work|labo(u)?r)', '(employ|work|labo(u)?r)(.*)floor'],'CompanyEntity'),
    'headquarters/office':(['headquarters', 'office'],'GlobalLocation'),
    'factory':(['factor(ies|y)','manufactur'],'GlobalLocation'),
    'migrant worker':(['migra(n)?t(.*)(employ|work|labo(u)?r)', '(employ|work|labo(u)?r)(.*)migra(n)?t'],'GlobalLocation'),
    'mobile worker':(['mobile(.*)(employ|work|labo(u)?r)', '(employ|work|labo(u)?r)(.*)mobile'],'GlobalLocation'),
    'supply chain':(['supply chain', 'supplier'],'SupplyChain'),
    'value chain':(['value chain'],'SupplyChain'),
    'confidential':(['confidential'],'Qualities'),
    'mandatory':(['mandatory'],'Qualities'),
    'regular':(['regular'],'Qualities'),
    'third-party':(['third( |-)party'],'Qualities'),
    'equal':(['equal'],'Qualities'),
    'anonymous':(['anonymous'],'Qualities'),
    'formal':(['formal'],'Qualities'),
    'written':(['written'],'Qualities'),
    'independent':(['independent'],'Qualities'),
    '-conscious/friendly/focused':(['-conscious', '-friendly', '-focus'],'Qualities'),
    '-balanced/neutral/agnostic':(['-balance', '-neutral', 'agnostic'],'Qualities'),
    'coronavirus':(['coronavirus', ' covid', 'global pandemic'],'COVID-19'),
    'black lives matter':(['black lives matter', 'blm', 'george floyd'],'BLM'),
    'blm-police':(['police (reform|violence)', 'defund the police', ' acab '],'BLM'),
    'criminal justice':(['criminal justice'],'BLM')
}

modifier_list = []
for key, val in modifier_dict.items():
  modifier_list.append(key)

## (SLIGHT MODIFICATION): Create output dataframe w/ indicator and summary columns
The file `'demo_example_input.csv'` will be loaded from the working directory, where it should be placed.

- MODIFY-1: Change the `cols_name_with_text` assignment to include all the column names in your dataframe that include your text data.
- MODIFY-2: Set the size of your batch (per iteration) to a reasonable number of rows based on how long each text is. This ensures that progress is saved incrementally before a Colab session times out.  
- MODIFY-3: Set the filepath to your working directory where the `*merge.csv` files should be saved
- MODIFY-4: Reload the output dataframe if necessary
- MODIFY-5: Change the indices depending on what column order you want for the final csv
- MODIFY-6: Insert the `ANY_*` indicator columns for practice term, outcome term, DEI context term, and practice-outcome term co-occurrence into `full_master_df` at column indices you prefer for the column order of the final CSV.

In [ ]:
input_df = pd.read_csv('demo_example_input.csv')

### Run the pattern search on the input dataframe

In [ ]:
input_string_col_name = 'total_text_lower'

#MODIFY-1: add all the column names with your text data
input_df[input_string_col_name] = input_df['Article Title'].str.lower() + ' ' + input_df['Article Excerpt'].str.lower()

In [ ]:
# Chunked processing
CHUNK_SIZE = 100 #MODIFY-2: set the size of your batch

start_row = 0 #if necessary, set to a nonzero row to resume from that row
process_df = input_df.iloc[start_row:,:]

quotient_floor = divmod(len(process_df),CHUNK_SIZE)[0]
remainder = divmod(len(process_df),CHUNK_SIZE)[1]

# All full CHUNK_SIZE-chunks
for i in range(quotient_floor):
  df_subset = process_df.iloc[i*CHUNK_SIZE:(i+1)*CHUNK_SIZE,:]
  tqdm.pandas()
  ind_df_subset = df_subset.progress_apply(lambda x: check_columns(x, input_string_col_name), axis=1)
  ind_df_subset.to_csv(f'{datetime.datetime.today().month}_{datetime.datetime.today().day}-DEI-Output-{(i+(start_row/CHUNK_SIZE))*CHUNK_SIZE}-{(i+1+(start_row/CHUNK_SIZE))*CHUNK_SIZE}_merge.csv', index=False)
  print('Completed rows ',(i+(start_row/CHUNK_SIZE))*CHUNK_SIZE,'to ',(i+1+(start_row/CHUNK_SIZE))*CHUNK_SIZE)

# Remainder <CHUNK_SIZE chunk
df_subset_final = process_df.iloc[quotient_floor*CHUNK_SIZE:,:]
tqdm.pandas()
ind_df_subset_final = df_subset_final.progress_apply(lambda x: check_columns(x, input_string_col_name), axis=1)
ind_df_subset_final.to_csv(f'{datetime.datetime.today().month}_{datetime.datetime.today().day}-DEI-Output-{(quotient_floor+(start_row/CHUNK_SIZE))*CHUNK_SIZE}-{(quotient_floor+(start_row/CHUNK_SIZE))*CHUNK_SIZE+remainder}_merge.csv', index=False)
print('\nCompleted rows ',(quotient_floor+(start_row/CHUNK_SIZE))*CHUNK_SIZE,'to ',(quotient_floor+(start_row/CHUNK_SIZE))*CHUNK_SIZE+remainder)

100%|██████████| 10/10 [09:16<00:00, 55.64s/it]


Completed rows  0.0 to  10.0


In [ ]:
# Compiling CSVs
working_dir_filepath = '/content/drive/MyDrive/Work/Rights-Colab-SS/Github Demo' #MODIFY-3: set the filepath to your working directory
files = os.path.join(working_dir_filepath, '*merge.csv')
all_filenames = glob.glob(files)
ind_df = pd.concat(map(pd.read_csv, all_filenames), ignore_index=True)

In [ ]:
ind_df.head()

,PRACTICE_TERMS_FOUND,OUTCOME_TERMS_FOUND,DEI-CONTEXT_TERMS_FOUND,ANY_RRP_TOPIC,ANY_WVP_TOPIC,ANY_DEISG_TOPIC,ANY_MBOD_TOPIC,ANY_WD_TOPIC,ANY_PE_TOPIC,ANY_SC_TOPIC,ANY_BS_TOPIC,ANY_CI_TOPIC,ANY_WSP_TOPIC,ANY_CA_TOPIC,ANY_PA_TOPIC,ANY_GO_TOPIC,ANY_GP_TOPIC,policy-fair dismissal_PRACTICE_Worker-Voice/Protection,compensation-(un)fair_PRACTICE_Worker-Security/Precarity,due diligence_PRACTICE_General-Practice,impacts/externalities-environmental_OUTCOME_Community-Action,life insurance_PRACTICE_Benefits/Services,controversy_OUTCOME_DEI-Strategy/Governance,hate speech accountability-advertising_PRACTICE_Public-Advocacy,disability coverage_PRACTICE_Benefits/Services,recruitment fee_PRACTICE_General-Practice,number of employee grievances resolved_OUTCOME_Worker-Voice/Protection,inclusive culture_OUTCOME_Worker-Voice/Protection,discrimination_OUTCOME_DEI-Strategy/Governance,absenteeism rate_OUTCOME_Recruitment/Retention/Promotion-Rates,front pay_OUTCOME_General-Outcome,quit/resign_OUTCOME_Recruitment/Retention/Promotion-Rates,policy-working hour_PRACTICE_Worker-Voice/Protection,worker union_PRACTICE_Worker-Voice/Protection,due diligence-risk identification/assessment process_PRACTICE_DEI-Strategy/Governance,acknowledgement-environmental justice_PRACTICE_Public-Advocacy,new hires diversity_OUTCOME_Recruitment/Retention/Promotion-Rates,court-ordered relief_OUTCOME_DEI-Strategy/Governance,policy-decent jobs_PRACTICE_Worker-Voice/Protection,report employee policy_PRACTICE_Worker-Voice/Protection,emergency back-up child/elder care services_PRACTICE_Benefits/Services,track employee engagement/satisfaction_PRACTICE_Worker-Voice/Protection,employer affinity program/group_PRACTICE_Workforce-Development,dress code_PRACTICE_Worker-Voice/Protection,programs/initiatives/policies-talent attract_PRACTICE_Recruitment/Retention/Promotion-Rates,community development_PRACTICE_Community-Action,background check_PRACTICE_Recruitment/Retention/Promotion-Rates,access to secure employment_OUTCOME_Worker-Security/Precarity,change in share value_OUTCOME_General-Outcome,report percent change-pay equity_PRACTICE_Pay-Equity,offshoring_PRACTICE_General-Practice,divest-customer_OUTCOME_Customer-Inclusiveness,employee feedback system_PRACTICE_Worker-Voice/Protection,track DEI goals-internal_PRACTICE_DEI-Strategy/Governance,track DEI performance-internal_PRACTICE_DEI-Strategy/Governance,client support and feedback system_PRACTICE_Customer-Inclusiveness,outsource_PRACTICE_General-Practice,transition in a disrupted industry_PRACTICE_General-Practice,written/formal contracts_PRACTICE_Worker-Security/Precarity,non-disclosure agreements_PRACTICE_Worker-Voice/Protection,harassment-local community_OUTCOME_Community-Action,composition-leadership_OUTCOME_Management/Board/Owner-Demographics,strike/walk-out-employee_OUTCOME_DEI-Strategy/Governance,misconduct_OUTCOME_DEI-Strategy/Governance,reject applicant_PRACTICE_Recruitment/Retention/Promotion-Rates,DEI roles-internal_PRACTICE_DEI-Strategy/Governance,highest paid ratio of pay_OUTCOME_Pay-Equity,stakeholder engagement_PRACTICE_Community-Action,line manager_PRACTICE_Worker-Voice/Protection,policy-local sourcing_PRACTICE_Supply-Chain,percent change-promotions_OUTCOME_Recruitment/Retention/Promotion-Rates,access to social security_PRACTICE_Worker-Voice/Protection,programs/initiatives/policy-work-life balance_PRACTICE_Benefits/Services,arbitration_PRACTICE_General-Practice,protest-local community_OUTCOME_Community-Action,wage dispute_OUTCOME_Pay-Equity,programs/initiatives-stakeholder engagement_PRACTICE_Community-Action,hire goal/quota-employee_PRACTICE_DEI-Strategy/Governance,policy-diverse representation_PRACTICE_Worker-Voice/Protection,disruption/delay-operational/workplace_OUTCOME_General-Outcome,partnership-local community_PRACTICE_Community-Action,to be liable_OUTCOME_General-Outcome,government action_OUTCOME_General-Outcome,retaliation/reprisal_OUTCOME_Worker-Voice/Protection,retirement provision_PRACTICE_Benefits/Services,pr

In [ ]:
ind_df.to_csv(f'{datetime.datetime.today().month}_{datetime.datetime.today().day}-DEI-OutputDF.csv', index=False)

### Compile final dataframe

In [ ]:
#MODIFY-4: Change the filename to reload the output dataframe if necessary
# ind_df = pd.read_csv('[DATE]-DEI-OutputDF.csv')

In [ ]:
# Put all the columns in one master df

#MODIFY-5: change the indices depending on what column order you want for the final csv
order_1 = input_df[input_df.columns[0:3]]
order_2 = ind_df[ind_df.columns[0:3]]
order_3 = ind_df[ind_df.columns[438:439]] #'MODIFIER_TERMS_FOUND' column
order_4 = input_df[input_df.columns[3:]]
order_5 = ind_df[ind_df.columns[3:438]]
order_6 = ind_df[ind_df.columns[439:]]
full_master_df = pd.concat([order_1,order_2,order_3,order_4,order_5,order_6], axis=1)
full_master_df.head()

,Article Title,Article Excerpt,URL Link,PRACTICE_TERMS_FOUND,OUTCOME_TERMS_FOUND,DEI-CONTEXT_TERMS_FOUND,MODIFIER_TERMS_FOUND,Date,Publication,total_text_lower,ANY_RRP_TOPIC,ANY_WVP_TOPIC,ANY_DEISG_TOPIC,ANY_MBOD_TOPIC,ANY_WD_TOPIC,ANY_PE_TOPIC,ANY_SC_TOPIC,ANY_BS_TOPIC,ANY_CI_TOPIC,ANY_WSP_TOPIC,ANY_CA_TOPIC,ANY_PA_TOPIC,ANY_GO_TOPIC,ANY_GP_TOPIC,policy-fair dismissal_PRACTICE_Worker-Voice/Protection,compensation-(un)fair_PRACTICE_Worker-Security/Precarity,due diligence_PRACTICE_General-Practice,impacts/externalities-environmental_OUTCOME_Community-Action,life insurance_PRACTICE_Benefits/Services,controversy_OUTCOME_DEI-Strategy/Governance,hate speech accountability-advertising_PRACTICE_Public-Advocacy,disability coverage_PRACTICE_Benefits/Services,recruitment fee_PRACTICE_General-Practice,number of employee grievances resolved_OUTCOME_Worker-Voice/Protection,inclusive culture_OUTCOME_Worker-Voice/Protection,discrimination_OUTCOME_DEI-Strategy/Governance,absenteeism rate_OUTCOME_Recruitment/Retention/Promotion-Rates,front pay_OUTCOME_General-Outcome,quit/resign_OUTCOME_Recruitment/Retention/Promotion-Rates,policy-working hour_PRACTICE_Worker-Voice/Protection,worker union_PRACTICE_Worker-Voice/Protection,due diligence-risk identification/assessment process_PRACTICE_DEI-Strategy/Governance,acknowledgement-environmental justice_PRACTICE_Public-Advocacy,new hires diversity_OUTCOME_Recruitment/Retention/Promotion-Rates,court-ordered relief_OUTCOME_DEI-Strategy/Governance,policy-decent jobs_PRACTICE_Worker-Voice/Protection,report employee policy_PRACTICE_Worker-Voice/Protection,emergency back-up child/elder care services_PRACTICE_Benefits/Services,track employee engagement/satisfaction_PRACTICE_Worker-Voice/Protection,employer affinity program/group_PRACTICE_Workforce-Development,dress code_PRACTICE_Worker-Voice/Protection,programs/initiatives/policies-talent attract_PRACTICE_Recruitment/Retention/Promotion-Rates,community development_PRACTICE_Community-Action,background check_PRACTICE_Recruitment/Retention/Promotion-Rates,access to secure employment_OUTCOME_Worker-Security/Precarity,change in share value_OUTCOME_General-Outcome,report percent change-pay equity_PRACTICE_Pay-Equity,offshoring_PRACTICE_General-Practice,divest-customer_OUTCOME_Customer-Inclusiveness,employee feedback system_PRACTICE_Worker-Voice/Protection,track DEI goals-internal_PRACTICE_DEI-Strategy/Governance,track DEI performance-internal_PRACTICE_DEI-Strategy/Governance,client support and feedback system_PRACTICE_Customer-Inclusiveness,outsource_PRACTICE_General-Practice,transition in a disrupted industry_PRACTICE_General-Practice,written/formal contracts_PRACTICE_Worker-Security/Precarity,non-disclosure agreements_PRACTICE_Worker-Voice/Protection,harassment-local community_OUTCOME_Community-Action,composition-leadership_OUTCOME_Management/Board/Owner-Demographics,strike/walk-out-employee_OUTCOME_DEI-Strategy/Governance,misconduct_OUTCOME_DEI-Strategy/Governance,reject applicant_PRACTICE_Recruitment/Retention/Promotion-Rates,DEI roles-internal_PRACTICE_DEI-Strategy/Governance,highest paid ratio of pay_OUTCOME_Pay-Equity,stakeholder engagement_PRACTICE_Community-Action,line manager_PRACTICE_Worker-Voice/Protection,policy-local sourcing_PRACTICE_Supply-Chain,percent change-promotions_OUTCOME_Recruitment/Retention/Promotion-Rates,access to social security_PRACTICE_Worker-Voice/Protection,programs/initiatives/policy-work-life balance_PRACTICE_Benefits/Services,arbitration_PRACTICE_General-Practice,protest-local community_OUTCOME_Community-Action,wage dispute_OUTCOME_Pay-Equity,programs/initiatives-stakeholder engagement_PRACTICE_Community-Action,hire goal/quota-employee_PRACTICE_DEI-Strategy/Governance,policy-diverse representation_PRACTICE_Worker-Voice/Protection,disruption/delay-operational/workplace_OUTCOME_General-Outcome,partnership-local community_PRACTICE_Community-Action,to be liable_OUTCOME_General-Outcome,government action_OUTCOME_General-Outcome,retaliati

In [ ]:
# Adding indicators of having any practice term or outcome term,
# to quickly identify events with co-occurrences of practice and outcome terms

# indicator format: {term}_{PRACTICE/OUTCOME/DEI-context term}_{category}

all_cols = list(full_master_df.columns)
practice_term_cols = []
outcome_term_cols = []
DEI_context_term_cols = []
modifier_term_cols = []

for col in all_cols:

    try:
        col_type = col.split('_')[1]

        if col_type == 'PRACTICE':
            practice_term_cols.append(col)
        elif col_type == 'OUTCOME':
            outcome_term_cols.append(col)
        elif col_type == 'DEI-CONTEXT':
            DEI_context_term_cols.append(col)
        elif col_type == 'MODIFIER':
            modifier_term_cols.append(col)

    except:
        continue


In [ ]:
full_master_df['ANY_DEI-CONTEXT_TERM'] = 0
for DEI_context_term_col in DEI_context_term_cols:
  full_master_df['ANY_DEI-CONTEXT_TERM'] = np.where(full_master_df[DEI_context_term_col] == 1,
                                                                    1, full_master_df['ANY_DEI-CONTEXT_TERM'])

full_master_df['ANY_PRACTICE_TERM'] = 0
for practice_term_col in practice_term_cols:
  full_master_df['ANY_PRACTICE_TERM'] = np.where(full_master_df[practice_term_col] == 1,
                                                                    1, full_master_df['ANY_PRACTICE_TERM'])

full_master_df['ANY_OUTCOME_TERM'] = 0
for outcome_term_col in outcome_term_cols:
  full_master_df['ANY_OUTCOME_TERM'] = np.where(full_master_df[outcome_term_col] == 1,
                                                                    1, full_master_df['ANY_OUTCOME_TERM'])

# Only practice + outcome co-occurence
full_master_df['ANY_PRACTICE_AND_OUTCOME'] = np.where((full_master_df['ANY_PRACTICE_TERM'] == 1) & (full_master_df['ANY_OUTCOME_TERM'] == 1), 1, 0)

In [ ]:
#MODIFY-6: Insert the indicator columns into `full_master_df` at column indices you prefer for the column order of the final CSV

for i, col in enumerate(['ANY_PRACTICE_AND_OUTCOME'
                         ]):
    mid = full_master_df[col]
    full_master_df.drop(labels=[col], axis=1, inplace = True)
    full_master_df.insert(i+3, col, mid) #MODIFY-6a: if necessary, change the index (set by 'i+3') to get the column order you want

for i, col in enumerate(['ANY_PRACTICE_TERM',
                         'ANY_OUTCOME_TERM',
                         'ANY_DEI-CONTEXT_TERM',
                         ]):
    mid = full_master_df[col]
    full_master_df.drop(labels=[col], axis=1, inplace = True)
    full_master_df.insert(i+25, col, mid) #MODIFY-6b: if necessary, change the index (set by 'i+25') to get the column order you want

In [ ]:
full_master_df.head()

,Article Title,Article Excerpt,URL Link,ANY_PRACTICE_AND_OUTCOME,PRACTICE_TERMS_FOUND,OUTCOME_TERMS_FOUND,DEI-CONTEXT_TERMS_FOUND,MODIFIER_TERMS_FOUND,Date,Publication,total_text_lower,ANY_RRP_TOPIC,ANY_WVP_TOPIC,ANY_DEISG_TOPIC,ANY_MBOD_TOPIC,ANY_WD_TOPIC,ANY_PE_TOPIC,ANY_SC_TOPIC,ANY_BS_TOPIC,ANY_CI_TOPIC,ANY_WSP_TOPIC,ANY_CA_TOPIC,ANY_PA_TOPIC,ANY_GO_TOPIC,ANY_GP_TOPIC,ANY_PRACTICE_TERM,ANY_OUTCOME_TERM,ANY_DEI-CONTEXT_TERM,policy-fair dismissal_PRACTICE_Worker-Voice/Protection,compensation-(un)fair_PRACTICE_Worker-Security/Precarity,due diligence_PRACTICE_General-Practice,impacts/externalities-environmental_OUTCOME_Community-Action,life insurance_PRACTICE_Benefits/Services,controversy_OUTCOME_DEI-Strategy/Governance,hate speech accountability-advertising_PRACTICE_Public-Advocacy,disability coverage_PRACTICE_Benefits/Services,recruitment fee_PRACTICE_General-Practice,number of employee grievances resolved_OUTCOME_Worker-Voice/Protection,inclusive culture_OUTCOME_Worker-Voice/Protection,discrimination_OUTCOME_DEI-Strategy/Governance,absenteeism rate_OUTCOME_Recruitment/Retention/Promotion-Rates,front pay_OUTCOME_General-Outcome,quit/resign_OUTCOME_Recruitment/Retention/Promotion-Rates,policy-working hour_PRACTICE_Worker-Voice/Protection,worker union_PRACTICE_Worker-Voice/Protection,due diligence-risk identification/assessment process_PRACTICE_DEI-Strategy/Governance,acknowledgement-environmental justice_PRACTICE_Public-Advocacy,new hires diversity_OUTCOME_Recruitment/Retention/Promotion-Rates,court-ordered relief_OUTCOME_DEI-Strategy/Governance,policy-decent jobs_PRACTICE_Worker-Voice/Protection,report employee policy_PRACTICE_Worker-Voice/Protection,emergency back-up child/elder care services_PRACTICE_Benefits/Services,track employee engagement/satisfaction_PRACTICE_Worker-Voice/Protection,employer affinity program/group_PRACTICE_Workforce-Development,dress code_PRACTICE_Worker-Voice/Protection,programs/initiatives/policies-talent attract_PRACTICE_Recruitment/Retention/Promotion-Rates,community development_PRACTICE_Community-Action,background check_PRACTICE_Recruitment/Retention/Promotion-Rates,access to secure employment_OUTCOME_Worker-Security/Precarity,change in share value_OUTCOME_General-Outcome,report percent change-pay equity_PRACTICE_Pay-Equity,offshoring_PRACTICE_General-Practice,divest-customer_OUTCOME_Customer-Inclusiveness,employee feedback system_PRACTICE_Worker-Voice/Protection,track DEI goals-internal_PRACTICE_DEI-Strategy/Governance,track DEI performance-internal_PRACTICE_DEI-Strategy/Governance,client support and feedback system_PRACTICE_Customer-Inclusiveness,outsource_PRACTICE_General-Practice,transition in a disrupted industry_PRACTICE_General-Practice,written/formal contracts_PRACTICE_Worker-Security/Precarity,non-disclosure agreements_PRACTICE_Worker-Voice/Protection,harassment-local community_OUTCOME_Community-Action,composition-leadership_OUTCOME_Management/Board/Owner-Demographics,strike/walk-out-employee_OUTCOME_DEI-Strategy/Governance,misconduct_OUTCOME_DEI-Strategy/Governance,reject applicant_PRACTICE_Recruitment/Retention/Promotion-Rates,DEI roles-internal_PRACTICE_DEI-Strategy/Governance,highest paid ratio of pay_OUTCOME_Pay-Equity,stakeholder engagement_PRACTICE_Community-Action,line manager_PRACTICE_Worker-Voice/Protection,policy-local sourcing_PRACTICE_Supply-Chain,percent change-promotions_OUTCOME_Recruitment/Retention/Promotion-Rates,access to social security_PRACTICE_Worker-Voice/Protection,programs/initiatives/policy-work-life balance_PRACTICE_Benefits/Services,arbitration_PRACTICE_General-Practice,protest-local community_OUTCOME_Community-Action,wage dispute_OUTCOME_Pay-Equity,programs/initiatives-stakeholder engagement_PRACTICE_Community-Action,hire goal/quota-employee_PRACTICE_DEI-Strategy/Governance,policy-diverse representation_PRACTICE_Worker-Voice/Protection,disruption/delay-operational/workplace_OUTCOME_General-Outcome,partnership-local community_PRACTICE_Community-Action,to be l

### (OPTIONAL): Checking some features of the final dataframe, some are TVL-specific

In [ ]:
# Total articles
full_master_df.shape[0]

10

In [ ]:
# Number of ARTICLES with a practice-outcome co-occurrence
# (Of course, need to check later if these co-occurrences make sense with our
# contexts, but just to get an idea)
full_master_df['ANY_PRACTICE_AND_OUTCOME'].sum()

9

##(OPTIONAL): Function to randomly generate CSV of positive samples
- Run this code if you want to create a separate CSV of positive samples for faster viewing or validation.

Slight modifications will be necessary:
- MODIFY-1: Change the `100` to whatever size sample you want
- MODIFY-2: Change the condition by which you are determining positivity if necessary. This code is presently capturing the rows that have a practice-outcome co-occurrence.
- MODIFY-3: Change the name of the CSV you are printing out to reflect the characteristics of your sample



In [ ]:
# Create validation sample
SAMPLE_SIZE = 100 #MODIFY-1: change 100 to whatever size sample you want
i = 0
rows_ind = []
while i < SAMPLE_SIZE:
  row_no = random.randrange(len(full_master_df.index))
  if full_master_df.iloc[row_no]['ANY_PRACTICE_AND_OUTCOME'] == 1: #MODIFY-2: change to column(s) you want to filter on (to get positive results)
    if row_no not in rows_ind:
      rows_ind.append(row_no)
      i += 1

DEI_valid_100 = full_master_df.iloc[rows_ind]
DEI_valid_100.to_csv(f'{datetime.datetime.today().month}_{datetime.datetime.today().day}-SAMPLE100-DEI.csv', index=False) #MODIFY-3: change 100 to whatever size sample you want

## (SLIGHT MODIFICATION): Create CSV(s)
Output dataframe `full_master_df` as the final CSV.
- You can change the name of the CSV you are printing out to reflect the characteristics of your CSV



In [ ]:
full_master_df.to_csv(f'{datetime.datetime.today().month}_{datetime.datetime.today().day}-DEI.csv', index=False)